---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025

# NBJ Élève – Accuracy & Loss (Fashion-MNIST)
Travail guidé avec questions à chaque étape.

## 1. Imports
### Questions
1. À quoi servent les bibliothèques chargées ici ?
2. Pourquoi fixe-t-on une graine aléatoire (`seed`) ?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
np.random.seed(42)

## 2. Chargement du dataset
### Questions
3. Combien y a‑t‑il d’images d’entraînement ?
4. Combien d’images de test ?
5. Quelle est la taille des images ?

In [ ]:
(fashion_x_train, fashion_y_train), (fashion_x_test, fashion_y_test) = keras.datasets.fashion_mnist.load_data()
print(fashion_x_train.shape, fashion_x_test.shape)

## 3. Filtrage de 3 classes : T-shirt (0), Sandale (5), Sac (8)
### Questions
6. Pourquoi choisit‑on seulement 3 classes ?
7. Pourquoi faut-il remapper les labels en (0,1,2) ?
8. Combien d’images restent après filtrage ?

In [ ]:
classes=[0,5,8]
def filtrer(x,y,classes):
    mask=np.isin(y,classes)
    x2=x[mask]; y2=y[mask]
    remap={c:i for i,c in enumerate(classes)}
    y2=np.vectorize(remap.get)(y2)
    return x2,y2

x_train,y_train=filtrer(fashion_x_train,fashion_y_train,classes)
x_test,y_test=filtrer(fashion_x_test,fashion_y_test,classes)
print(x_train.shape,y_train.shape)

## 4. Visualisation
### Questions
9. Que représentaient les six images affichées ?
10. Certaines classes semblent-elles plus difficiles à reconnaître ? Pourquoi ?
11. Les images sont-elles détaillées ou limitées ? Quel effet sur l’apprentissage ?

In [ ]:
plt.figure(figsize=(8,4))
for i in range(6):
    idx=np.random.randint(0,len(x_train))
    plt.subplot(2,3,i+1)
    plt.imshow(x_train[idx],cmap="gray")
    plt.axis("off")
plt.show()

## 5. Préparation des données
### Questions
12. Pourquoi diviser les valeurs par 255 ?
13. Pourquoi ajouter une dimension `(1)` ?
14. Que représente cette dimension supplémentaire ?

In [ ]:
x_train = x_train.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0
x_train = np.expand_dims(x_train,-1)
x_test  = np.expand_dims(x_test,-1)
print(x_train.shape)

## 6. Construction du CNN
### Questions
15. Combien y a‑t‑il de couches convolutionnelles ?
16. À quoi sert `MaxPooling2D` ?
17. Pourquoi la dernière couche possède-t-elle 3 neurones ?
18. Que fait `softmax` ?
19. Pourquoi utilise-t-on `sparse_categorical_crossentropy` comme loss ?

In [ ]:
def creer_cnn():
    model=keras.Sequential([
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64,(3,3),activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128,activation='relu'),
        layers.Dense(3,activation='softmax')
    ])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

model=creer_cnn()
model.summary()

## 7. Entraînement court (5 epochs)
### Questions
20. Que signifient les valeurs affichées pendant l’entraînement ?
21. Comment évoluent la loss et l’accuracy ?
22. Les valeurs train et validation sont-elles proches ? Que signifie cet écart ?

In [ ]:
hist_court=model.fit(x_train,y_train,epochs=5,batch_size=64,validation_split=0.2)

## 8. Courbes (5 epochs)
### Questions
23. Quelle courbe est la plus basse : loss train ou val ?
24. Quelle courbe est la plus haute : accuracy train ou val ?
25. Que peut-on conclure du comportement du modèle après 5 epochs ?

In [ ]:
def tracer(h):
    d=h.history; e=range(1,len(d['loss'])+1)
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1); plt.plot(e,d['loss']); plt.plot(e,d['val_loss']); plt.title("Loss")
    plt.subplot(1,2,2); plt.plot(e,d['accuracy']); plt.plot(e,d['val_accuracy']); plt.title("Accuracy")
    plt.show()

tracer(hist_court)

## 9. Entraînement long (30 epochs)
### Questions
26. Comment évolue la loss d’entraînement quand on augmente le nombre d’epochs ?
27. À partir de quand la loss de validation commence-t-elle à remonter ?
28. Comment évolue l’accuracy de validation ?

In [ ]:
model2=creer_cnn()
hist_long=model2.fit(x_train,y_train,epochs=30,batch_size=64,validation_split=0.2)

## 10. Courbes (30 epochs)
### Questions
29. Observe la loss de validation : repère un début d’overfitting.
30. Observe l’accuracy de validation : stagne-t-elle ? baisse-t-elle ?
31. Comment choisir un bon nombre d’epochs ?

In [ ]:
tracer(hist_long)

## 11. Évaluation finale
### Questions
32. Quel modèle (5 ou 30 epochs) a la meilleure performance sur le test ?
33. Le surentraînement peut-il réduire la performance finale ? Pourquoi ?

In [ ]:
print("Modèle 5 epochs :", model.evaluate(x_test,y_test,verbose=0))
print("Modèle 30 epochs :", model2.evaluate(x_test,y_test,verbose=0))

## 12. Prédictions
### Questions
34. Le modèle se trompe-t-il parfois ?
35. Pourquoi une bonne accuracy ne garantit-elle pas l’absence d’erreurs ?

In [ ]:
indices=np.random.randint(0,len(x_test),size=4)
for idx in indices:
    img=x_test[idx]
    pred=model2.predict(img[np.newaxis,...],verbose=0)
    lab=np.argmax(pred[0])
    plt.imshow(img.squeeze(),cmap="gray")
    plt.title(f"Prédit : {lab}")
    plt.axis("off")
    plt.show()

---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

--- 2025